Aim:  converting data from here (https://github.com/microsoft/RoadDetections/?tab=readme-ov-file) into shapefiles to upload to Google Earth Engine (GEE)

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import os

TSV to CSV 
- TSV: tab seperated file format - native format for microsoft roads data
- CSV: comma separated file format. Chosen as GEE allows upload up to 10 GB isnteast of 2 GB for shapefiles

Format differences: 
- TSV: has ISO3 in one column and what appears to be a GeoJSON data in another column 
- CSV: geometry stored as a column and properties in other columns    

In [ ]:
import json
import pandas as pd

def convert_geojson_column_to_csv(
    input_path: str,
    output_path: str = "parsed_output.csv",
    input_format: str = "tsv",
    max_rows: int = None
):
    """
    Converts a CSV/TSV with ISO3 and GeoJSON column to a flat CSV with geometry and properties.

    Parameters:
    - input_path (str): Path to the input file (CSV or TSV).
    - output_path (str): Path to save the parsed CSV.
    - input_format (str): Either 'csv' or 'tsv'.
    - max_rows (int or None): Max number of rows to process. Use None to process all rows.
    """
    def parse_line(json_str):
        json_str = json_str.replace('""', '"').strip('"')
        data = json.loads(json_str)
        coordinates = data["geometry"]["coordinates"]
        geometry_type = data["geometry"]["type"]
        properties = data["properties"]

        if geometry_type == "LineString":
            coords_wkt = ", ".join([f"{x[0]} {x[1]}" for x in coordinates])
            geometry = f"LINESTRING ({coords_wkt})"
        else:
            geometry = "UNKNOWN"

        return {
            # "iso3": iso3,
            "geometry": geometry,
            **properties
        }

    # === Load input file ===
    sep = '\t' if input_format == "tsv" else ','
    df_raw = pd.read_csv(input_path, sep=sep, header=None, names=["iso3", "geojson_str"])

    if max_rows is not None:
        df_raw = df_raw.head(max_rows)

    # === Parse all rows ===
    parsed_rows = [parse_line(row.geojson_str) for _, row in df_raw.iterrows()]
    df_parsed = pd.DataFrame(parsed_rows)

    # === Save to CSV ===
    df_parsed.to_csv(output_path, index=False)
    print(f"✅ Parsed {len(df_parsed)} rows and saved to '{output_path}'.")


import pandas as pd
import os
import math

def split_large_csv(input_file, output_prefix, num_chunks=2, chunk_size=None, rows_estimate=None):
    """
    Split a large CSV file into multiple smaller files.
    
    Parameters:
    ----------
    input_file : str
        Path to the input CSV file
    output_prefix : str
        Prefix for the output files, will be followed by _part1, _part2, etc.
    num_chunks : int
        Number of chunks to split the file into (default: 2)
    chunk_size : int, optional
        Size of each chunk in rows (overrides num_chunks if provided)
    rows_estimate : int, optional
        Estimated number of rows in the file (to avoid counting)
        
    Returns:
    -------
    list
        List of paths to the created output files
    """
    print(f"Starting to split {input_file}")
    output_files = []
    
    # Get the file size for progress reporting
    file_size = os.path.getsize(input_file)
    print(f"File size: {file_size / (1024 * 1024 * 1024):.2f} GB")
    
    # Determine total rows if not provided
    if not rows_estimate:
        print("Counting rows in the file (this may take a while)...")
        with open(input_file, 'r', encoding='utf-8') as f:
            rows_estimate = sum(1 for _ in f) - 1  # Subtract 1 for header
        print(f"Found {rows_estimate} rows")
    else:
        print(f"Using estimated row count: {rows_estimate}")
    
    # Calculate rows per chunk
    if chunk_size:
        rows_per_chunk = chunk_size
        estimated_chunks = math.ceil(rows_estimate / rows_per_chunk)
        print(f"Will create approximately {estimated_chunks} chunks with {rows_per_chunk} rows each")
    else:
        rows_per_chunk = math.ceil(rows_estimate / num_chunks)
        print(f"Will create {num_chunks} chunks with approximately {rows_per_chunk} rows each")
    
    # Read and write in chunks
    print("Starting to write chunks...")
    chunks_written = 0
    current_chunk = 1
    
    # Use pandas to process in chunks
    for chunk in pd.read_csv(input_file, chunksize=100000):  # Process 100k rows at a time
        # Determine which output file this chunk belongs to
        output_file = f"{output_prefix}_part{current_chunk}.csv"
        
        # If this is a new file, write with header
        if output_file not in output_files:
            chunk.to_csv(output_file, index=False, mode='w')
            output_files.append(output_file)
            print(f"Created new file: {output_file}")
        else:
            # Append to existing file without header
            chunk.to_csv(output_file, index=False, mode='a', header=False)
        
        # Update counters
        chunks_written += len(chunk)
        
        # Progress reporting
        if chunks_written % 1000000 == 0:
            print(f"Processed {chunks_written:,} rows ({chunks_written/rows_estimate*100:.1f}%)")
        
        # Check if we need to move to the next file
        if chunks_written >= current_chunk * rows_per_chunk and current_chunk < num_chunks:
            current_chunk += 1
    
    print(f"Successfully split {input_file} into {len(output_files)} files")
    print(f"Output files: {output_files}")
    return output_files



In [ ]:
in_folder = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\raw\roads\microsoft"

output_folder = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom" 


In [ ]:
# Using os.path.join for proper path construction
import os
input_file_path = os.path.join(in_folder, "Caribbean", "Caribbean.tsv")
output_path_csv = os.path.join(output_folder, "Caribbean.csv")



List folders in the Microsoft roads data directory

In [ ]:
import fnmatch

# List all items in the directory 
country_folders = os.listdir(in_folder)

# using fnmatch for more complex wildcard patterns

# remove those with .zip extension
country_folders = [f for f in country_folders if not fnmatch.fnmatch(f, '*.zip')]

#if you want to remove specific folders, you can use patterns like:
country_folders = [f for f in country_folders if not fnmatch.fnmatch(f, '*Northern_America*')]
# country_folders = [f for f in country_folders if not fnmatch.fnmatch(f, '*Eastern_Africa*')]

country_folders

Get list of countries that need processing (have TSV but no CSV yet)

In [ ]:

countries_to_process = []
for each in country_folders:
    input_file_path = os.path.join(in_folder, each, f"{each}.tsv")
    output_path_csv = os.path.join(output_folder, f"{each}.csv")
    
    # Check if input exists but output doesn't
    if os.path.exists(input_file_path) and not os.path.exists(output_path_csv):
        countries_to_process.append(each)
        print(f"Will process: {each}")
    elif not os.path.exists(input_file_path):
        print(f"Input file not found: {input_file_path}. Skipping.")
    else:
        print(f"Output already exists for {each}. Skipping.")

print(f"Found {len(countries_to_process)} countries to process out of {len(country_folders)}")


Process only the countries that need it

In [ ]:
# Convert the GeoJSON column to CSV
# Loop through the countries to process
for each in countries_to_process:
    # Construct the paths (no need for checking again)
    input_file_path = os.path.join(in_folder, each, f"{each}.tsv")
    output_path_csv = os.path.join(output_folder, f"{each}.csv")
    
    # Convert the TSV to CSV
    try:
        print(f"Converting {each}...")
        convert_geojson_column_to_csv(input_file_path, output_path_csv)
        print(f"Conversion complete for {each}")
    except Exception as e:
        print(f"Error converting {each}: {str(e)}")

print("All processing complete!")

Split large files i.e., CSVs over 10GB (only needed to split N America into two parts)

In [11]:
# input_csv = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom\Northern_America.csv"
# output_prefix = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom\Northern_America_split"

input_csv = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom\Eastern_Europe.csv"
output_prefix = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom\Eastern_Europe"

# Split into 2 chunks using the estimated row count of 70 million
output_files = split_large_csv(
    input_file=input_csv,
    output_prefix=output_prefix,
    num_chunks=3,
    # rows_estimate=15000000
)

Starting to split C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom\Eastern_Europe.csv
File size: 3.51 GB
Counting rows in the file (this may take a while)...
Found 22992884 rows
Will create 3 chunks with approximately 7664295 rows each
Starting to write chunks...
Created new file: C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom\Eastern_Europe_part1.csv
Processed 1,000,000 rows (4.3%)
Processed 2,000,000 rows (8.7%)
Processed 3,000,000 rows (13.0%)
Processed 4,000,000 rows (17.4%)
Processed 5,000,000 rows (21.7%)
Processed 6,000,000 rows (26.1%)
Processed 7,000,000 rows (30.4%)
Created new file: C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\microsoft\processed_csvs_w_geom\Eastern_Europe_part2.